In [25]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

from tqdm.auto import tqdm 

import torch 
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
import torch.optim as optim 

from torchvision.models import resnet50 , ResNet50_Weights


In [26]:
# class basicallly for extracting the feature maps from the pretrained model Resnet50
class resnet_feature_extractor(torch.nn.Module):
    def __init__(self):
        super(resnet_feature_extractor,self).__init__()
        self.model = resnet50(weights=ResNet50_Weights.DEFAULT)

        self.model.eval()
        for p in self.model.parameters():
            p.requires_grad = False


        #hook to extract the feature maps

        def hook(module, input ,output)-> None:
            # this hook saves the extracted features map on self.forward
            self.features.append(output.detach())

        self.model.layer2[-1].register_forward_hook(hook)
        self.model.layer3[-1].register_forward_hook(hook)

    def forward(self,input):
        self.features=[]
        with torch.no_grad():  # torch.no_grad() is for the It temporarily turns off gradient calculation, meaning: No .grad is stored , No computation graph is built , Much lower memory use Faster inference
            _=self.model(input)

        self.avg=torch.nn.AvgPool2d(3,stride=1)
        fmap_size=self.features[0].shape[-2] #feature map sizes h,w 
        self.resize = torch.nn.AdaptiveAvgPool2d(fmap_size)

        resized_maps=[self.resize(self.avg(fmap)) for fmap in self.features]
        patch=torch.cat(resized_maps,1)

        return patch

In [27]:
import torch
import torch.nn.functional as F
from torchvision.models import resnet50, ResNet50_Weights

class ResNetFeatureExtractorFast(torch.nn.Module):
    def __init__(self, pretrained=True, device=None):
        super().__init__()
        weights = ResNet50_Weights.DEFAULT if pretrained else None
        self.model = resnet50(weights=weights)
        self.model.eval()
        for p in self.model.parameters():
            p.requires_grad = False

        # Register hooks once (optional) or you can call layers directly (see below)
        self.features = []
        self.handles = []
        self.handles.append(self.model.layer2[-1].register_forward_hook(self._hook))
        self.handles.append(self.model.layer3[-1].register_forward_hook(self._hook))

        if device is not None:
            self.to(device)

    def _hook(self, module, input, output):
        # keep on-device detached copy
        self.features.append(output.detach())

    def forward(self, x, target_spatial=None):
        # clear
        self.features = []

        # run backbone (no grad)
        with torch.no_grad():
            _ = self.model(x)

        if len(self.features) == 0:
            raise RuntimeError("No features captured. Are hooks registered?")

        # target spatial default = layer2 size
        if target_spatial is None:
            h = self.features[0].shape[-2]
            w = self.features[0].shape[-1]
            target_spatial = (h, w)

        resized = []
        for fmap in self.features:
            # smooth (keep same size)
            fmap_smoothed = F.avg_pool2d(fmap, kernel_size=3, stride=1, padding=1)
            # resize to target spatial using interpolate (works on MPS and CPU)
            if fmap_smoothed.shape[-2:] != target_spatial:
                fmap_resized = F.interpolate(fmap_smoothed, size=target_spatial,
                                             mode='bilinear', align_corners=False)
            else:
                fmap_resized = fmap_smoothed
            resized.append(fmap_resized)

        patch = torch.cat(resized, dim=1)
        return patch

    def remove_hooks(self):
        for h in self.handles:
            h.remove()
        self.handles = []


In [28]:
import torch.nn.functional as F

def extract_patch_no_hooks(resnet_model, x, target_spatial=None):
    # follow ResNet forward up to layer3, return concat(layer2, layer3) resized to layer2 size
    x = resnet_model.conv1(x)
    x = resnet_model.bn1(x)
    x = resnet_model.relu(x)
    x = resnet_model.maxpool(x)
    x = resnet_model.layer1(x)
    out2 = resnet_model.layer2(x)        # (B,512,H2,W2)
    out3 = resnet_model.layer3(out2)     # (B,1024,H3,W3)

    if target_spatial is None:
        target_spatial = (out2.shape[-2], out2.shape[-1])

    fmap2 = F.avg_pool2d(out2, kernel_size=3, stride=1, padding=1)
    fmap3 = F.avg_pool2d(out3, kernel_size=3, stride=1, padding=1)
    if fmap3.shape[-2:] != target_spatial:
        fmap3 = F.interpolate(fmap3, size=target_spatial, mode='bilinear', align_corners=False)

    patch = torch.cat([fmap2, fmap3], dim=1)  # (B,1536,H2,W2)
    return patch


In [29]:
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

extractor = ResNetFeatureExtractor(pretrained=True, device=device)
extractor.model.eval()
for p in extractor.model.parameters():
    p.requires_grad = False

# sample batch
x = torch.randn(2,3,224,224).to(device)
patch = extractor(x)   # shape (2,1536, H_ref, W_ref) e.g. (2,1536,28,28)
print(patch.shape)

# when done in notebook
# extractor.remove_hooks()


torch.Size([2, 1536, 28, 28])


### Now I replaced AdaptiveAvgPool2d with F.interpolate because AdaptiveAvgPool2d has an MPS bug when the output size doesn't divide input size. interpolate is robust and supported on MPS.



In [30]:
import torch, os
from pathlib import Path
from torch.utils.data import DataLoader, Subset
import torch.optim as optim
import torch.nn.functional as F
from tqdm.auto import tqdm

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print("Device:", device)

CAEMODEL_DIR = Path("saved_spatial_caes_simple")
CAEMODEL_DIR.mkdir(exist_ok=True)


Device: mps


In [31]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN,
                         std=IMAGENET_STD)
])
train_ds = ImageFolder("mvtec_all/train", transform=transform)  # folder containing class-subfolders
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=4)

val_ds = ImageFolder("mvtec_all/val", transform=transform)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=4)


In [32]:
# train_ds must be ImageFolder("mvtec_all/train", transform=transform) created earlier
from torchvision.datasets import ImageFolder

train_ds = ImageFolder("mvtec_all/train", transform=transform)

category_list = train_ds.classes

def loader_for_category_from_full(train_ds, cat_name, batch_size=8, shuffle=True, num_workers=0):
    idx = train_ds.class_to_idx[cat_name]
    inds = [i for i, (_, l) in enumerate(train_ds.samples) if l == idx]
    subset = Subset(train_ds, inds)
    return DataLoader(subset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)


In [33]:
import torch.nn as nn

class FeatCAE(nn.Module):
    """Autoencoder."""

    def __init__(self, in_channels=1000, latent_dim=50, is_bn=True):
        super(FeatCAE, self).__init__()

        layers = []
        layers += [nn.Conv2d(in_channels, (in_channels + 2 * latent_dim) // 2, kernel_size=1, stride=1, padding=0)]
        if is_bn:
            layers += [nn.BatchNorm2d(num_features=(in_channels + 2 * latent_dim) // 2)]
        layers += [nn.ReLU()]
        layers += [nn.Conv2d((in_channels + 2 * latent_dim) // 2, 2 * latent_dim, kernel_size=1, stride=1, padding=0)]
        if is_bn:
            layers += [nn.BatchNorm2d(num_features=2 * latent_dim)]
        layers += [nn.ReLU()]
        layers += [nn.Conv2d(2 * latent_dim, latent_dim, kernel_size=1, stride=1, padding=0)]

        self.encoder = nn.Sequential(*layers)

        # if 1x1 conv to reconstruct the rgb values, we try to learn a linear combination
        # of the features for rgb
        layers = []
        layers += [nn.Conv2d(latent_dim, 2 * latent_dim, kernel_size=1, stride=1, padding=0)]
        if is_bn:
            layers += [nn.BatchNorm2d(num_features=2 * latent_dim)]
        layers += [nn.ReLU()]
        layers += [nn.Conv2d(2 * latent_dim, (in_channels + 2 * latent_dim) // 2, kernel_size=1, stride=1, padding=0)]
        if is_bn:
            layers += [nn.BatchNorm2d(num_features=(in_channels + 2 * latent_dim) // 2)]
        layers += [nn.ReLU()]
        layers += [nn.Conv2d((in_channels + 2 * latent_dim) // 2, in_channels, kernel_size=1, stride=1, padding=0)]
        # layers += [nn.ReLU()]

        self.decoder = nn.Sequential(*layers)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [20]:
def train_cae_for_category_simple(cat_name,
                                  extractor,
                                  train_ds,
                                  in_channels=1536,
                                  latent_dim=100,
                                  epochs=15,
                                  batch_size=8,
                                  lr=1e-3,
                                  device=device,
                                  save_dir=CAEMODEL_DIR):
    print(f"=== Train CAE for {cat_name} ===")
    loader = loader_for_category_from_full(train_ds, cat_name, batch_size=batch_size, shuffle=True, num_workers=0)

    # instantiate CAE and optimizer
    cae = FeatCAE(in_channels=in_channels, latent_dim=latent_dim, is_bn=True).to(device).float()
    optimizer = optim.Adam(cae.parameters(), lr=lr)
    criterion = torch.nn.MSELoss()

    extractor = extractor.to(device)
    extractor.model.eval()
    for p in extractor.model.parameters():
        p.requires_grad = False

    best_loss = float('inf')
    for epoch in range(1, epochs+1):
        cae.train()
        running = 0.0
        n = 0
        pbar = tqdm(loader, desc=f"{cat_name} Epoch {epoch}/{epochs}", leave=False)
        for imgs, _ in pbar:
            imgs = imgs.to(device)

            # extract features on-the-fly (backbone frozen)
            with torch.no_grad():
                features = extractor(imgs)    # (B,1536,H,W) on device

            # forward CAE
            optimizer.zero_grad()
            outputs = cae(features)          # must be same shape as features
            loss = criterion(outputs, features)
            loss.backward()
            optimizer.step()

            running += loss.item() * imgs.size(0)
            n += imgs.size(0)
            pbar.set_postfix({"batch_loss": loss.item()})

        epoch_loss = running / max(1, n)
        print(f"{cat_name} Epoch {epoch}/{epochs} loss: {epoch_loss:.6f}")

        # save best
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            save_path = Path(save_dir) / f"cae_{cat_name}.pth"
            torch.save(cae.state_dict(), save_path)
            print(f"Saved best CAE -> {save_path} (loss {best_loss:.6f})")

    return cae


In [34]:

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
extractor = ResNetFeatureExtractorFast(pretrained=True, device=device)
extractor.model.eval()
for p in extractor.model.parameters():
    p.requires_grad = False

# smoke test (ensure it captures features and returns the expected shape)
x = torch.randn(1,3,224,224).to(device)
with torch.no_grad():
    patch = extractor(x)
print("extractor patch shape:", patch.shape)  # expect (1,1536, H, W), e.g. (1,1536,28,28)

model = FeatCAE(in_channels=1536, latent_dim=100).to(device)
extractor = extractor.to(device)


extractor patch shape: torch.Size([1, 1536, 28, 28])


In [35]:
# tune these hyperparams as needed
EPOCHS = 15
BATCH_SIZE = 8
LR = 1e-3
LATENT_DIM = 100

for cat in category_list:
    ckpt = CAEMODEL_DIR / f"cae_{cat}.pth"
    if ckpt.exists():
        print("Skipping (exists):", cat)
        continue
    _ = train_cae_for_category_simple(cat,
                                      extractor=extractor,
                                      train_ds=train_ds,
                                      in_channels=1536,
                                      latent_dim=LATENT_DIM,
                                      epochs=EPOCHS,
                                      batch_size=BATCH_SIZE,
                                      lr=LR,
                                      device=device,
                                      save_dir=CAEMODEL_DIR)


=== Train CAE for bottle ===


bottle Epoch 1/15:   0%|          | 0/27 [00:00<?, ?it/s]

bottle Epoch 1/15 loss: 0.256879
Saved best CAE -> saved_spatial_caes_simple/cae_bottle.pth (loss 0.256879)


bottle Epoch 2/15:   0%|          | 0/27 [00:00<?, ?it/s]

bottle Epoch 2/15 loss: 0.054569
Saved best CAE -> saved_spatial_caes_simple/cae_bottle.pth (loss 0.054569)


...

zipper Epoch 13/15 loss: 0.020063
Saved best CAE -> saved_spatial_caes_simple/cae_zipper.pth (loss 0.020063)


zipper Epoch 14/15:   0%|          | 0/30 [00:00<?, ?it/s]

zipper Epoch 14/15 loss: 0.019352
Saved best CAE -> saved_spatial_caes_simple/cae_zipper.pth (loss 0.019352)


zipper Epoch 15/15:   0%|          | 0/30 [00:00<?, ?it/s]

zipper Epoch 15/15 loss: 0.018721
Saved best CAE -> saved_spatial_caes_simple/cae_zipper.pth (loss 0.018721)


In [36]:
# 1) confirm extractor exists and has handles registered
print("extractor exists:", 'extractor' in globals())
print("handles attr present?", hasattr(extractor, "handles"))
print("num handles:", len(getattr(extractor, "handles", [])))
print("handles objects:", getattr(extractor, "handles", []))

# 2) show the modules we expect hooks on
print("layer2[-1]:", extractor.model.layer2[-1])
print("layer3[-1]:", extractor.model.layer3[-1])

# 3) quick forward test (random input)
x = torch.randn(1,3,224,224).to(device)
extractor.features = []    # clear
with torch.no_grad():
    _ = extractor(x)
print("features captured:", len(extractor.features))
if len(extractor.features)>0:
    for i,f in enumerate(extractor.features):
        print(i, f.shape, f.device)


extractor exists: True
handles attr present? True
num handles: 2
handles objects: [<torch.utils.hooks.RemovableHandle object at 0x308243250>, <torch.utils.hooks.RemovableHandle object at 0x30857fc50>]
layer2[-1]: Bottleneck(
  (conv1): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
)
layer3[-1]: Bottleneck(
  (conv1): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(256, 256, kernel_size=(3,

In [37]:
trained_caes = {}
for cat in category_list:
    p = CAEMODEL_DIR / f"cae_{cat}.pth"
    if not p.exists():
        print("Missing CAE for", cat); continue
    cae = FeatCAE(in_channels=1536, latent_dim=LATENT_DIM, is_bn=True)
    cae.load_state_dict(torch.load(p, map_location=device))
    cae.to(device).eval()
    trained_caes[cat] = cae
print("Loaded CAEs:", list(trained_caes.keys()))


Loaded CAEs: ['bottle', 'cable', 'capsule', 'carpet', 'grid', 'hazelnut', 'leather', 'metal_nut', 'pill', 'screw', 'tile', 'toothbrush', 'transistor', 'wood', 'zipper']
